In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [20]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

324
735
[29, 56, 8, 1, 82, 9, 45, 22, 24, 67]
[56, 8, 1, 82, 9, 45, 22, 24, 67, 2]


In [21]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [22]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [23]:
# def sam(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [24]:
# hyper-parameters
epochs = 60000
hidden_layer = 128
inp_out_size = 324
learning_rate = 0.001
num_steps = 13

In [25]:
tf.reset_default_graph()
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.random_normal([hidden_layer, inp_out_size], mean=0, stddev=0.001))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
print tf.trainable_variables()

[<tensorflow.python.ops.variables.Variable object at 0x7fec9996bc90>, <tensorflow.python.ops.variables.Variable object at 0x7fec980baa10>, <tensorflow.python.ops.variables.Variable object at 0x7fec980ba9d0>, <tensorflow.python.ops.variables.Variable object at 0x7fec980d4610>, <tensorflow.python.ops.variables.Variable object at 0x7fec980e44d0>]


In [26]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [27]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [28]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [29]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
ix = 0
smooth_loss = -np.log(1.0 / vocab_size)*num_steps

In [31]:
print epochs
h = np.zeros(hidden_layer)
for i in range(epochs):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
    num_steps = num_steps - 1 if i % 2 == 0 else num_steps + 1 # variable length sequences
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, h, _ = sess.run([loss, states, optimize_op], {a: x, b: y, initial: h}) 
    smooth_loss = smooth_loss * 0.999 + l * 0.001
    if i % 1000 == 0:
        print 'epoch {0}, loss = {1}'.format(i, smooth_loss)
#         print sess.run(Whh)[0][:30]
    ix += num_steps
        
#         generate(sess,40)

60000
epoch 0, loss = 74.1500637822
epoch 1000, loss = 71.4036227919
epoch 2000, loss = 68.7669468156
epoch 3000, loss = 65.1413892697
epoch 4000, loss = 62.8374848909
epoch 5000, loss = 61.8369479722
epoch 6000, loss = 61.3870267922
epoch 7000, loss = 61.1182143732
epoch 8000, loss = 60.8779046606
epoch 9000, loss = 60.5381072472
epoch 10000, loss = 60.0850656331
epoch 11000, loss = 59.5457284149
epoch 12000, loss = 58.9689993363
epoch 13000, loss = 58.3150665894
epoch 14000, loss = 57.5510557253
epoch 15000, loss = 56.7005705248
epoch 16000, loss = 55.6742156223
epoch 17000, loss = 54.4605514208
epoch 18000, loss = 53.0528639365
epoch 19000, loss = 51.4728144186
epoch 20000, loss = 49.6241414483
epoch 21000, loss = 47.5471605026
epoch 22000, loss = 45.4261219301
epoch 23000, loss = 43.4088280672
epoch 24000, loss = 41.4144884494
epoch 25000, loss = 39.4337976094
epoch 26000, loss = 37.4555986506
epoch 27000, loss = 35.534961589
epoch 28000, loss = 33.5491288219
epoch 29000, loss = 31

In [32]:
generate(sess, 500)

of Torment taught her 
 How to frown and how to chide ; —thou 
 Through 's winged Fancy roam , 
 From with tasting 'd 'd : 
 She has winged self-overaw every , 
 Fades a meet in every mind : 
 not 
 gaz 's winged Fancy roam 
 Doth 
 Through 's winged Fancy roam , 
 From with tasting 'd straw moment , 
 Or will bring self-overaw 's mind 
 Doth not hear in every new roam : 
 not with tasting 'd straw stealth : 
 She will gaz 'd 's Fancy ; 
 Cloys , winged Fancy : 
 She a winged self-overaw every , 
 Fades will meet self-overaw every mind : 
 not with gaz every straw The , 
 Or a winged Fancy every muffled , 
 Or will meet self-overaw every mind : 
 
 much gaz 'd at home , 
 
 not , tasting ever home : 
 Or will bring self-overaw 's mind : 
 Too much gaz 'd at stealth : 
 She will bring self-overaw every mind : 
 
 much gaz 'd at home . 
 
 
 much 's winged Fancy home 
 She has tasting clasp every at home 
 Sweet 
 golden 's winged Fancy home : 
 The for tasting 'd 'd : 
 She will meet se

In [33]:
sess.run(Whh)

array([[ 0.03073823,  0.16024236,  0.16885728, ...,  0.05288818,
        -0.07688183, -0.03028169],
       [-0.24785835, -0.04390815,  0.1450949 , ..., -0.05311445,
        -0.01250314,  0.11281139],
       [-0.33347312, -0.20284222,  0.23098749, ..., -0.10504574,
        -0.28402022,  0.03956333],
       ..., 
       [ 0.05662825,  0.16807128,  0.00710124, ...,  0.22454964,
        -0.06467728,  0.10287257],
       [-0.06466194, -0.10157995, -0.11470366, ...,  0.09235065,
         0.3563292 ,  0.02229096],
       [ 0.08362083,  0.13572359,  0.1389745 , ...,  0.22135787,
         0.00691832,  0.11428044]], dtype=float32)